<a href="https://colab.research.google.com/github/THUNDERSAMA/cnn_furniture/blob/main/furniture_detect_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available()else 'cpu')
print(device)

cuda


In [ ]:
train_path="/content/drive/MyDrive/img/train"
tr_transformer=transforms.Compose([transforms.Resize((150,150))
,transforms.ToTensor()])
tr_dataset=torchvision.datasets.ImageFolder(train_path,transform=tr_transformer)
train_loader=torch.utils.data.DataLoader(dataset=tr_dataset,batch_size=12,shuffle=False)

In [ ]:
def get_mean_std(loader):
    mean=0.
    std=0.
    total_images_count=0
    for images,_ in loader:
        image_count=images.size(0)
        images=images.view(image_count,images.size(1),-1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += image_count
    mean /= total_images_count
    std /= total_images_count

    return mean,std
get_mean_std(train_loader)

(tensor([0.6090, 0.5617, 0.5238]), tensor([0.2307, 0.2434, 0.2494]))

In [ ]:
transformer=transforms.Compose([transforms.Resize((150,150))
,transforms.RandomHorizontalFlip(),
transforms.ToTensor(),
transforms.Normalize([0.2307, 0.2434, 0.2494],[0.2307, 0.2434, 0.2494])
])

In [ ]:
train_path="/content/drive/MyDrive/img/train"
test_path="/content/drive/MyDrive/img/val/mix"
train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=50,shuffle =True)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=50,shuffle =True
)

In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['bed', 'chair', 'sofa', 'swivelchair', 'table']


In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()



        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)

        self.relu1=nn.ReLU()

        self.pool=nn.MaxPool2d(kernel_size=2)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()



        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)




        output=output.view(-1,32*75*75)


        output=self.fc(output)

        return output

In [ ]:
model=ConvNet(num_classes=5).to(device)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=10
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

4024 423


In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):

    #Evaluation and training of dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        if (torch.cuda.is_available()):
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count



    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if (torch.cuda.is_available()):
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Saving the model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy


Epoch: 0 Train Loss: tensor(1.7995) Train Accuracy: 0.7837972166998012 Test Accuracy: 7.612293144208038
Epoch: 1 Train Loss: tensor(0.8233) Train Accuracy: 0.8635685884691849 Test Accuracy: 8.709219858156029
Epoch: 2 Train Loss: tensor(0.6131) Train Accuracy: 0.8866799204771372 Test Accuracy: 8.50354609929078
Epoch: 3 Train Loss: tensor(0.2897) Train Accuracy: 0.9259443339960238 Test Accuracy: 9.044917257683215
Epoch: 4 Train Loss: tensor(0.2648) Train Accuracy: 0.9333996023856859 Test Accuracy: 8.6903073286052
Epoch: 5 Train Loss: tensor(0.1636) Train Accuracy: 0.9607355864811133 Test Accuracy: 9.286052009456265
Epoch: 6 Train Loss: tensor(0.1573) Train Accuracy: 0.9637176938369781 Test Accuracy: 9.281323877068559
Epoch: 7 Train Loss: tensor(0.0800) Train Accuracy: 0.9781312127236581 Test Accuracy: 9.300236406619385
Epoch: 8 Train Loss: tensor(0.1016) Train Accuracy: 0.9744035785288271 Test Accuracy: 9.361702127659575
Epoch: 9 Train Loss: tensor(0.1429) Train Accuracy: 0.9617296222664

In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()



        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()

        self.pool=nn.MaxPool2d(kernel_size=2)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.relu2=nn.ReLU()



        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)



    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)




        output=output.view(-1,32*75*75)


        output=self.fc(output)

        return output


In [ ]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=5)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=5, bias=True)
)

In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.2307, 0.2434, 0.2494],[0.2307, 0.2434, 0.2494])
])

In [ ]:
def prediction(img_path,transformer):

    image=Image.open(img_path)

    image_tensor=transformer(image).float()


    image_tensor=image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    input=Variable(image_tensor)


    output=model(input)

    index=output.data.numpy().argmax()

    pred=classes[index]

    return pred


In [ ]:
images_path=glob.glob(test_path+'/*.jpg')

In [ ]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [ ]:
pred_dict

{'00000943.jpg': 'bed',
 '00000948.jpg': 'sofa',
 '00000945.jpg': 'bed',
 '00000947.jpg': 'bed',
 '00000944.jpg': 'bed',
 '00000949.jpg': 'bed',
 '00000942.jpg': 'bed',
 '00000946.jpg': 'bed',
 '00000047.jpg': 'bed',
 '00000048.jpg': 'swivelchair',
 '00000012.jpg': 'chair',
 '00000045.jpg': 'chair',
 '00000055.jpg': 'chair',
 '00000050.jpg': 'chair',
 '00000052.jpg': 'chair',
 '00000051.jpg': 'chair',
 '00000212.jpg': 'sofa',
 '00000206.jpg': 'sofa',
 '00000211.jpg': 'sofa',
 '00000209.jpg': 'sofa',
 '00000216.jpg': 'sofa',
 '00000214.jpg': 'sofa',
 '00000210.jpg': 'table',
 '00000213.jpg': 'sofa',
 '00000215.jpg': 'sofa',
 '00000398.jpg': 'table',
 '00000401.jpg': 'table',
 '00000399.jpg': 'table',
 '00000400.jpg': 'table',
 '00000396.jpg': 'bed',
 '00000397.jpg': 'bed',
 '00000019.jpg': 'swivelchair',
 '00000074.jpg': 'swivelchair',
 '00000077.jpg': 'swivelchair',
 '00000073.jpg': 'swivelchair',
 '00000075.jpg': 'sofa',
 '00000078.jpg': 'swivelchair',
 '00000076.jpg': 'swivelchair',
